# Improved Chest X-Ray Pipeline

This notebook implements a pipeline designed to outperform the baseline CheXNet. 

**Key Improvements:**
1.  **Architecture**: EfficientNet-V2-S (Newer, faster, and more accurate than DenseNet).
2.  **Loss Function**: Focal Loss (to handle class imbalance better than BCE).
3.  **Optimizer**: AdamW with Cosine Annealing Scheduler (better convergence).
4.  **Augmentation**: Stronger augmentations (RandomAffine, ColorJitter).
5.  **Resolution**: 320x320 (Higher resolution for better detail).

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Check device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Data Preparation (Enhanced)

In [ ]:
# Configuration
BASE_DIR = '.'
CSV_PATH = os.path.join(BASE_DIR, 'Data_Entry_2017_v2020.csv')
IMAGE_DIRS = [
    os.path.join(BASE_DIR, 'images'),
    os.path.join(BASE_DIR, 'images 2')
]
BATCH_SIZE = 16
IMG_SIZE = 320  # Increased resolution
EPOCHS = 10     # More epochs for better convergence
LEARNING_RATE = 1e-4

# Load Data
df = pd.read_csv(CSV_PATH)

# Map images to paths
image_paths = {}
for img_dir in IMAGE_DIRS:
    if os.path.exists(img_dir):
        for fname in os.listdir(img_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths[fname] = os.path.join(img_dir, fname)

df['path'] = df['Image Index'].map(image_paths)
df = df.dropna(subset=['path'])

# Process Labels
all_labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
              'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']

for label in all_labels:
    df[label] = df['Finding Labels'].map(lambda x: 1.0 if label in x else 0.0)

print(f"Total images: {len(df)}")

In [ ]:
# Train/Val Split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")

In [ ]:
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = all_labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = row['path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        label = torch.tensor([row[l] for l in self.labels], dtype=torch.float32)
        return image, label

# Enhanced Transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05)), # Added Affine
    transforms.ColorJitter(brightness=0.1, contrast=0.1), # Added Color Jitter
    transforms.ToTensor(),
    normalize
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    normalize
])

train_dataset = ChestXrayDataset(train_df, transform=train_transform)
val_dataset = ChestXrayDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

## 2. Improved Model (EfficientNet-V2-S)

In [ ]:
class ImprovedModel(nn.Module):
    def __init__(self, num_classes):
        super(ImprovedModel, self).__init__()
        # EfficientNet V2 Small - Better accuracy/parameter ratio than DenseNet
        self.backbone = models.efficientnet_v2_s(weights='DEFAULT')
        
        # Replace classifier head
        num_ftrs = self.backbone.classifier[1].in_features
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.backbone(x)

model = ImprovedModel(num_classes=len(all_labels)).to(device)

## 3. Advanced Training Components (Focal Loss & Scheduler)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        bce_loss = nn.functional.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Use Focal Loss instead of BCE
criterion = FocalLoss(gamma=2)

# AdamW Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)

# Cosine Annealing Scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels_np = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            
            all_preds.append(outputs.cpu().numpy())
            all_labels_np.append(labels.cpu().numpy())
            
    all_preds = np.concatenate(all_preds)
    all_labels_np = np.concatenate(all_labels_np)
    
    aucs = []
    for i in range(len(all_labels)):
        try:
            auc = roc_auc_score(all_labels_np[:, i], all_preds[:, i])
            aucs.append(auc)
        except ValueError:
            aucs.append(0.5)
            
    mean_auc = np.mean(aucs)
    return running_loss / len(loader.dataset), mean_auc

print("Starting training (Improved Pipeline)...")
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_auc = evaluate(model, val_loader, criterion)
    
    # Step the scheduler
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    print(f"Epoch {epoch+1}/{EPOCHS} [LR: {current_lr:.6f}] - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Mean AUC: {val_auc:.4f}")

torch.save(model.state_dict(), 'improved_model.pth')
print("Model saved to improved_model.pth")